In [1]:
import pandas as pd

from core import extract_embeddings, load_sentence_model, semantic_search
from core import load_actions_dataset, compute_effects_from_indicator, generate_policies_from_bills

In [2]:
# 1) Carregue o dataset de PLs/ações (precisa ter 'municipio', 'data_apresentacao', 'acao')
dataset = load_actions_dataset("data/dataset.npy")

emb = extract_embeddings(dataset)
model = load_sentence_model()

# 2) Carregue o indicador (mesma normalização de nomes de cidade usada no indicador)
indicator = pd.read_csv("data/criminal_indicator.csv")  # deve conter municipio_norm, ano, semestre, taxa_homicidios_100k

/home/thiago/anaconda3/envs/AI/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
matches = semantic_search(
    query="Como reduzir a taxa de homicídios no município?", 
    dataset=dataset, model=model, embeddings=emb, top_k=1000
)

In [4]:
# 3) Compute os efeitos (delta entre semestre atual e próximo)
effects = compute_effects_from_indicator(
    bills=matches,
    indicator_df=indicator,
    city_col="municipio_norm",
    value_col="taxa_homicidios_100k",
)

In [6]:
import numpy as np

# effects é uma lista de (municipio, descricao_do_pl, delta_indicador)
policies = generate_policies_from_bills(effects, min_group_members=2, similarity_threshold=0.75)

# 4) Ordene e inspecione
policies = sorted(policies, key=lambda x: x['quality_score'], reverse=True)

for p in policies:
    scores = np.array([row[2] for row in p['actions']])
    n =  scores.shape[0]
    wins = np.sum(scores < 0.0)

    if (wins / n) <= 0.5:
        continue

    print(f"(Score: {p['quality_score']:.3f} - Win Rate: {wins}/{n}) {p['policy']}")


(Score: 0.800 - Win Rate: 4/4) Criar programa de combate às pichações no município.
(Score: 0.750 - Win Rate: 3/3) Criar mecanismo de punição administrativa para combater a violência contra a mulher no município.
(Score: 0.750 - Win Rate: 3/3) Criar o programa “Medicamento em Casa” no município.
(Score: 0.667 - Win Rate: 2/2) Instituir o mês “Junho Violento” para conscientização e prevenção da violência contra pessoas idosas no município.
(Score: 0.667 - Win Rate: 2/2) Instituir a campanha “Alerta dengue” no município.
(Score: 0.667 - Win Rate: 6/8) Criar o programa “Doe um Ponto de ônibus” no município.
(Score: 0.667 - Win Rate: 2/2) Instituir o Prêmio “Advocacia Cidadã” no município.
(Score: 0.667 - Win Rate: 2/2) Criar o programa “Cuidando de Quem Cuida” no município.
(Score: 0.667 - Win Rate: 2/2) Criar câmara mirim no município.
(Score: 0.667 - Win Rate: 2/2) Instituir a campanha “Janeiro Branco” no município.
(Score: 0.667 - Win Rate: 2/2) Alterar o Plano Diretor do município.
(S